In [1]:
#import dependencies 
from numpy.random import seed
from numpy.random import randn
from scipy import stats
import numpy as np

import math  

import seaborn as sns
sns.set(style="darkgrid")
from matplotlib import pyplot as plt

import pandas as pd
import os

In [3]:
#check list of files in data 
print(os.listdir("./data/open_cloud"))

['m5d_open_cloud_20x_y-cruncher_11-14-2019.csv', 'm5d_open_cloud_30x_sysbench_11-14-2019.csv', 'm5d_open_cloud_30x_y-cruncher_11-14-2019.csv', 'm5d_open_cloud_20x_sysbench_11-14-2019.csv', 'm5d_open_cloud_30x_pgbench_11-14-2019.csv']


In [44]:
data_name = 'm5d_open_cloud_30x_pgbench_11-14-2019.csv'

In [45]:
df_data = pd.read_csv('./data/open_cloud/' + data_name);

In [46]:
df_data.head()

,instanceID,experimentID,instanceType,wallTime,clients,threads,scaleFactor,transactionsType,queryMode,duration,transactions,mountPoint
0,i-081386f91917e838e,1573768024-0,m5d.large,60.190190,10,10,1,TPC-B (sort of),simple,60 s,145271,data_directory = '/var/lib/postgresql/9.5/main'
1,i-081386f91917e838e,1573768024-0,m5d.large,60.187281,10,10,1,TPC-B (sort of),simple,60 s,145140,data_directory = '/var/lib/postgresql/9.5/main'
2,i-081386f91917e838e,1573768024-0,m5d.large,60.176184,10,10,1,TPC-B (sort of),simple,60 s,145455,data_directory = '/var/lib/postgresql/9.5/main'
3,i-081386f91917e838e,1573768024-0,m5d.large,60.177015,10,10,1,TPC-B (sort of),simple,60 s,145326,data_directory = '/var/lib/postgresql/9.5/main'
4,i-081386f91917e838e,1573768024-0,m5d.large,60.180249,10,10,1,TPC-B (sort of),simple,60 s,145640,data_directory = '/var/lib/postgresql/9.5/main'


In [47]:
df = df_data.groupby('instanceID').mean()

In [48]:
df

,wallTime,clients,threads,scaleFactor,transactions
instanceID,,,,,
i-00ae78d83e9b0a706,60.317044,10.0,10.0,1.0,142499.4
i-00ff625dd40e36264,60.270116,10.0,10.0,1.0,144412.5
i-015aae8f081e83dd1,60.257329,10.0,10.0,1.0,167268.4
i-0184f3296c5ced0a3,60.248411,10.0,10.0,1.0,144516.6
i-02318c82e163422e3,60.206323,10.0,10.0,1.0,33337.9
i-02333dfb92b0fd7c2,60.184761,10.0,10.0,1.0,163668.3
i-02839bf5d52dc440a,60.292868,10.0,10.0,1.0,143645.6
i-02baf91f487c27cd0,60.171360,10.0,10.0,1.0,143317.5
i-02bc03c5959e7d210,60.231698,10.0,10.0,1.0,144509.6


In [49]:
#if sysbench
#df.drop(columns =['thread-num'], inplace = True)

In [50]:
#print to csv
df.to_csv(path_or_buf='./data/summary/' + data_name + '-Summary.csv')

In [51]:
df.describe()

,wallTime,clients,threads,scaleFactor,transactions
count,30.000000,30.0,30.0,30.0,30.000000
mean,60.229738,10.0,10.0,1.0,135313.236667
std,0.045141,0.0,0.0,0.0,40778.073232
min,60.163577,10.0,10.0,1.0,33337.900000
25%,60.186309,10.0,10.0,1.0,143710.950000
50%,60.222895,10.0,10.0,1.0,144609.100000
75%,60.269127,10.0,10.0,1.0,152983.300000
max,60.317044,10.0,10.0,1.0,167268.400000


In [52]:
#graph the probablity of getting "better vm"